<a href="https://colab.research.google.com/github/bintualkassoum/candy-counter/blob/main/Fine_tune_and_test_candy_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Upload the exported images and annotations ZIP files (`train.zip` and `test.zip`) to the default directory (`/content`).

# Fine-tune and evaluate a candy detection model

See the tutorial [**Training Custom Object Detector**](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html) for more information and troubleshooting.

## 0. Upload scripts and set up Paths

First you need to upload the helper scripts `config.py` and `generate_tfrecord.py`. The former will make it easier to set up and keep track of the various paths and the latter converts images and annotations into a data format useful for training tensorflow models, `tfrecords`.

Upload `config.py` to the default Colab directory (`/content`).

Once that is done, run `config.py`:

In [11]:
#Install Packages 
import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [113]:
#@title Run Model Pre-Sets

def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


ALL_MODELS = {
'CenterNet HourGlass104 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1',
'CenterNet HourGlass104 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/hourglass_512x512_kpts/1',
'CenterNet HourGlass104 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024/1',
'CenterNet HourGlass104 Keypoints 1024x1024' : 'https://tfhub.dev/tensorflow/centernet/hourglass_1024x1024_kpts/1',
'CenterNet Resnet50 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1',
'CenterNet Resnet50 V1 FPN Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512_kpts/1',
'CenterNet Resnet101 V1 FPN 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1',
'CenterNet Resnet50 V2 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1',
'CenterNet Resnet50 V2 Keypoints 512x512' : 'https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512_kpts/1',
'EfficientDet D0 512x512' : 'https://tfhub.dev/tensorflow/efficientdet/d0/1',
'EfficientDet D1 640x640' : 'https://tfhub.dev/tensorflow/efficientdet/d1/1',
'EfficientDet D2 768x768' : 'https://tfhub.dev/tensorflow/efficientdet/d2/1',
'EfficientDet D3 896x896' : 'https://tfhub.dev/tensorflow/efficientdet/d3/1',
'EfficientDet D4 1024x1024' : 'https://tfhub.dev/tensorflow/efficientdet/d4/1',
'EfficientDet D5 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d5/1',
'EfficientDet D6 1280x1280' : 'https://tfhub.dev/tensorflow/efficientdet/d6/1',
'EfficientDet D7 1536x1536' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
'SSD MobileNet v2 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2',
'SSD MobileNet V1 FPN 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v1/fpn_640x640/1',
'SSD MobileNet V2 FPNLite 320x320' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1',
'SSD MobileNet V2 FPNLite 640x640' : 'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_640x640/1',
'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_640x640/1',
'SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)' : 'https://tfhub.dev/tensorflow/retinanet/resnet50_v1_fpn_1024x1024/1',
'SSD ResNet101 V1 FPN 640x640 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1',
'SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)' : 'https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_1024x1024/1',
'SSD ResNet152 V1 FPN 640x640 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_640x640/1',
'SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)' : 'https://tfhub.dev/tensorflow/retinanet/resnet152_v1_fpn_1024x1024/1',
'Faster R-CNN ResNet50 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1',
'Faster R-CNN ResNet50 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_1024x1024/1',
'Faster R-CNN ResNet50 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_800x1333/1',
'Faster R-CNN ResNet101 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_640x640/1',
'Faster R-CNN ResNet101 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1',
'Faster R-CNN ResNet101 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_800x1333/1',
'Faster R-CNN ResNet152 V1 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_640x640/1',
'Faster R-CNN ResNet152 V1 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1',
'Faster R-CNN ResNet152 V1 800x1333' : 'https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_800x1333/1',
'Faster R-CNN Inception ResNet V2 640x640' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1',
'Faster R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
'Mask R-CNN Inception ResNet V2 1024x1024' : 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'
}

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [3]:
%run /content/Tensorflow/config.py

By clicking **Files**, you should see the directory structure created.

Upload `generate_tfrecord.py` to `Tensorflow/scripts`

## 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [118]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
  !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection (TFOD)
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

Fix some other installation/version issues.

In [ ]:
!pip install tensorflow --upgrade
!pip uninstall protobuf matplotlib -y
!pip install protobuf==3.19 matplotlib==3.2
!pip install -U opencv-python

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Run the verification script to check that the installation is ok.

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
!wget {'/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'}
!mv {'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'+'.tar.gz'} {'/content/Tensorflow/workspace/training_demo/pre-trained-models'}
!cd {'/content/Tensorflow/workspace/training_demo/pre-trained-models'} && tar -zxvf {'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'+'.tar.gz'}

Restart runtime and re-run `config.py`.

In [10]:
# Re-run config.py after restarting the runtime.
%run /content/Tensorflow/config.py
import object_detection

## 2. Upload the label map, images and annotations.

Upload `label_map.pbtxt` to `Tensorflow/workspace/annotations`.

Upload the exported images and annotations ZIP files (`train.zip` and `test.zip`) to the default directory (`/content`).

In [16]:
from google.colab import files

uploaded = files.upload()

Saving test.zip to test.zip
Saving train.zip to train.zip


Unzip the uploaded `train.zip` and `text.zip`.

In [ ]:
!unzip 'train.zip' -d '/content/Tensorflow/workspace/images'
!unzip 'test.zip' -d '/content/Tensorflow/workspace/images'

## 3. Create TF records

Check the printed outupt and verify that the created `tfrecord` files both have more than 0 examples.

In [ ]:
!python 'Tensorflow/scripts/generate_tfrecord.py' -x 'Tensorflow/workspace/images/train/Annotations' -i 'Tensorflow/workspace/images/train/images' -l 'Tensorflow/workspace/annotations/label_map.pbtxt' -o 'Tensorflow/workspace/annotations/train.record'
!python 'Tensorflow/scripts/generate_tfrecord.py' -x 'Tensorflow/workspace/images/test/Annotations' -i 'Tensorflow/workspace/images/test/images' -l 'Tensorflow/workspace/annotations/label_map.pbtxt' -o 'Tensorflow/workspace/annotations/test.record'

## 4. Copy Model Config to Training Folder

In [64]:
!cp {os.path.join('/content/Tensorflow/workspace/pre-trained-models/', 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_pipeline.config')} {os.path.join('/content/Tensorflow/workspace/training_demo/pre-trained-models/read_checkpoints.py')}

## 5. Update Config For Transfer Learning

In [53]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [65]:
config = config_util.get_configs_from_pipeline_file('/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', config_override=None)

In [ ]:
config

In [67]:
from google.protobuf import text_format as pbtfimport

labelmap_path = os.path.join('/content/Tensorflow/workspace/training_demo/annotations', 'label_map.pbtxt')

def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}
    
    with open('/content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt', 'r') as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

labels = read_label_map(labelmap_path)    

In [68]:
print(labels)

{'Moon': 1, 'Insect': 2, 'Black_star': 3, 'Grey_star': 4, 'Unicorn_whole': 5, 'Unicorn_head': 6, 'Owl': 7, 'Cat': 8}


In [69]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile('/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [92]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join('/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'checkpoint')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = '/content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join('/content/Tensorflow/workspace/training_demo/annotations', 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = '/content/Tensorflow/workspace/annotations/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join('/content/Tensorflow/workspace/training_demo/annotations', 'test.record')]

In [93]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile('/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## 6. Train the model

In [94]:
TRAINING_SCRIPT = os.path.join('/content/Tensorflow/tensorflow/models', 'research', 'object_detection', 'model_main_tf2.py')

In [95]:
command = "python {} --model_dir={} --alsologtostderr --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint','/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config')

In [96]:
print(command)

python /content/Tensorflow/tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint --alsologtostderr --pipeline_config_path=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --num_train_steps=2000


In [ ]:
!{command}

## 7. Evaluate the Model

In [98]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint')

In [99]:
print(command)

python /content/Tensorflow/tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint --pipeline_config_path=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --checkpoint_dir=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint


In [ ]:
!{command}

# 8. Load a trained model from a checkpoint

## Build a detection model and load pre-trained model weights

Here we will choose which Object Detection model we will use.
Select the architecture and it will be loaded automatically.
If you want to change the model to try other architectures later, just change the next cell and execute following ones.

**Tip:** if you want to read more details about the selected model, you can follow the link (model handle) and read additional documentation on TF Hub. After you select a model, we will print the handle to make it easier.

In [101]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [102]:
#@title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'SSD MobileNet V2 FPNLite 320x320' # @param ['CenterNet HourGlass104 512x512','CenterNet HourGlass104 Keypoints 512x512','CenterNet HourGlass104 1024x1024','CenterNet HourGlass104 Keypoints 1024x1024','CenterNet Resnet50 V1 FPN 512x512','CenterNet Resnet50 V1 FPN Keypoints 512x512','CenterNet Resnet101 V1 FPN 512x512','CenterNet Resnet50 V2 512x512','CenterNet Resnet50 V2 Keypoints 512x512','EfficientDet D0 512x512','EfficientDet D1 640x640','EfficientDet D2 768x768','EfficientDet D3 896x896','EfficientDet D4 1024x1024','EfficientDet D5 1280x1280','EfficientDet D6 1280x1280','EfficientDet D7 1536x1536','SSD MobileNet v2 320x320','SSD MobileNet V1 FPN 640x640','SSD MobileNet V2 FPNLite 320x320','SSD MobileNet V2 FPNLite 640x640','SSD ResNet50 V1 FPN 640x640 (RetinaNet50)','SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)','SSD ResNet101 V1 FPN 640x640 (RetinaNet101)','SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)','SSD ResNet152 V1 FPN 640x640 (RetinaNet152)','SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)','Faster R-CNN ResNet50 V1 640x640','Faster R-CNN ResNet50 V1 1024x1024','Faster R-CNN ResNet50 V1 800x1333','Faster R-CNN ResNet101 V1 640x640','Faster R-CNN ResNet101 V1 1024x1024','Faster R-CNN ResNet101 V1 800x1333','Faster R-CNN ResNet152 V1 640x640','Faster R-CNN ResNet152 V1 1024x1024','Faster R-CNN ResNet152 V1 800x1333','Faster R-CNN Inception ResNet V2 640x640','Faster R-CNN Inception ResNet V2 1024x1024','Mask R-CNN Inception ResNet V2 1024x1024']
model_handle = ALL_MODELS[model_display_name]

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

Selected model:SSD MobileNet V2 FPNLite 320x320
Model Handle at TensorFlow Hub: https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1


## Loading the selected model from TensorFlow Hub

Here we just need the model handle that was selected and use the Tensorflow Hub library to load it to memory.


In [ ]:
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

## Loading an image

Let's try the model on a simple image. To help with this, we provide a list of test images.

Here are some simple things to try out if you are curious:
* Try running inference on your own images, just upload them to colab and load the same way it's done in the cell below.
* Modify some of the input images and see if detection still works.  Some simple things to try out here include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).

**Be careful:** when using images with an alpha channel, the model expect 3 channels images and the alpha will count as a 4th.



In [124]:
IMAGES_FOR_TEST = {
  'Image 1' : '/content/Tensorflow/workspace/training_demo/images/test/images/c8a6ad3b-cd_04.jpg',
  'Image 2' : '/content/Tensorflow/workspace/training_demo/images/test/images/f49b9d1f-cd_24.jpg'
}

In [ ]:
#@title Image Selection (don't forget to execute the cell!) { display-mode: "form"}
selected_image = 'Image 1' # @param ['Image 1', 'Image 2']
flip_image_horizontally = False #@param {type:"boolean"}
convert_image_to_grayscale = False #@param {type:"boolean"}

image_path = IMAGES_FOR_TEST[selected_image]
image_np = load_image_into_numpy_array(image_path)

# Flip horizontally
if(flip_image_horizontally):
  image_np[0] = np.fliplr(image_np[0]).copy()

# Convert image to grayscale
if(convert_image_to_grayscale):
  image_np[0] = np.tile(
    np.mean(image_np[0], 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()

# 9. Detect from an Image

In [172]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

Change the `image_path` to fit an image in the test set.

In [174]:
image_path = IMAGES_FOR_TEST[selected_image]

In [173]:
category_index = label_map_util.create_category_index_from_labelmap('/content/Tensorflow/workspace/training_demo/annotations/label_map.pbtxt')
category_index

{1: {'id': 1, 'name': 'Moon'},
 2: {'id': 2, 'name': 'Insect'},
 3: {'id': 3, 'name': 'Black_star'},
 4: {'id': 4, 'name': 'Grey_star'},
 5: {'id': 5, 'name': 'Unicorn_whole'},
 6: {'id': 6, 'name': 'Unicorn_head'},
 7: {'id': 7, 'name': 'Owl'},
 8: {'id': 8, 'name': 'Cat'}}

##Inference

In [187]:
# Running Inference
results = hub_model(image_np)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

dict_keys(['detection_multiclass_scores', 'raw_detection_boxes', 'num_detections', 'detection_scores', 'detection_anchor_indices', 'detection_boxes', 'detection_classes', 'raw_detection_scores'])


## Visualizing the results

Here is where we will need the TensorFlow Object Detection API to show the squares from the inference step (and the keypoints when available).

the full documentation of this method can be seen [here](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py)

Here you can, for example, set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [ ]:
label_id_offset = 1
image_np_with_detections = image_np.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in result:
  keypoints = result['detection_keypoints'][0]
  keypoint_scores = result['detection_keypoint_scores'][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_detections[0])
plt.show()

# 10. Freezing the Graph

In [247]:
FREEZE_SCRIPT = os.path.join('/content/Tensorflow/models', 'research', 'object_detection', 'exporter_main_v2.py')

In [248]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,'/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint', '/content/Tensorflow/models/research/object_detection')

In [249]:
print(command)

python /content/Tensorflow/models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --trained_checkpoint_dir=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint --output_directory=/content/Tensorflow/models/research/object_detection


In [ ]:
!{command}

# 12. Conversion to TFLite

In [251]:
TFLITE_SCRIPT = os.path.join('/content/Tensorflow/models', 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

In [252]:
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,'/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint', '/content/Tensorflow/models/research/object_detection')

In [253]:
print(command)

python /content/Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config --trained_checkpoint_dir=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint --output_directory=/content/Tensorflow/models/research/object_detection


In [ ]:
!{command}

In [268]:
FROZEN_TFLITE_PATH = os.path.join('/content/Tensorflow/models', 'research', 'object_detection', 'saved_model')
TFLITE_MODEL = os.path.join('/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint', '/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model', 'detect.tflite')

In [269]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,300,300,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL)

In [270]:
print(command)

tflite_convert --saved_model_dir=/content/Tensorflow/models/research/object_detection/saved_model --output_file=/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/detect.tflite --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [271]:
!{command}

2022-07-17 22:15:15.811853: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/training/saving/saveable_object_util.py", line 131, in restore
    self.handle_op, self._var_shape, restored_tensor)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/resource_variable_ops.py", line 308, in shape_safe_assign_variable_handle
    shape.assert_is_compatible_with(value_tensor.shape)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/tensor_shape.py", line 1291, in assert_is_compatible_with
    raise ValueError("Shapes %s and %s are incompatible" % (self, other))
ValueError: Shapes (1, 1, 128, 54) and (1, 1, 128, 546) are incompatible

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/tflite_conver

# 13. Zip and Export Models 

In [272]:
!tar -czf models.tar.gz {'/content/Tensorflow/workspace/training_demo/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint'}

tar: Removing leading `/' from member names
